In [158]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.support import expected_conditions as EC
import bs4
import re
from selenium.webdriver.common.keys import Keys

In [2]:
# adjust the functions for this website. 
suburbs= ("Sea Point",'Green Point',"Gardens")
def setup_driver():
    """"start the chrome driver and go to property24/com/to-rent
    """
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.implicitly_wait(5)
    driver.get('https://www.property24.com/to-rent')
    time.sleep(2)
    return driver
driver = setup_driver()



[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:01<00:00, 6.29MB/s]
<ipython-input-2-41c0e65daecb>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [165]:
def original_search(driver,suburb)->str:
    """Do the search of one suburb and return the Base Url for that suburbget base url for that specific suburb search"""
    # find the search bar
    search_bar = driver.find_element(By.XPATH,'//*[@id="token-input-AutoCompleteItems"]')
    time.sleep(2)
    def send_keys_delay(controller,keys,delay=0.1):
        """"send keys one at a time"""
        for key in keys:
            controller.send_keys(key)
            time.sleep(delay)

    # send keys
    send_keys_delay(search_bar,suburb)
    time.sleep(3)
    #click on first option
    driver.find_element(By.XPATH,'//*[@id="SearchBox"]/div/ul/li').click()
    time.sleep(2)
    #Click on search button
    driver.find_element(By.XPATH,'//*[@id="SearchBar"]/div[2]/div[1]/div/form/button').click()
    #Click on cookies button
    try:
        driver.find_element(By.XPATH,'//*[@id="cookieBannerClose"]').click()
    except:
        print("no Cookies button ")
        
    
    time.sleep(4)
    base_url = driver.current_url
    return base_url
    

In [5]:
base_url = original_search(driver,suburbs[0]) 
base_url

'https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021'

In [6]:
def make_soup(driver):
    
    soup = BeautifulSoup(driver.page_source,"html.parser")
    body_soup = soup.body
    return body_soup

In [7]:
body_soup = make_soup(driver)

In [ ]:
body_soup

In [181]:
for i in body_soup.find_all("div",{'class':"pull-right"}):
    print(i.text)



                Showing : 1 - 20
of 283










Download the App






In [193]:
def number_of_pages(body_soup:bs4.element.Tag) -> int:
    """"find the number of listed places in the search to figure out how many pages to crawl"""
    listingcount_options = body_soup.find_all("div",{'class':"pull-right"})
    for i in listingcount_options:
        print("string: ",i.text)
        if "Showing" in i.text:
            print("found showing", i)
            listingcount_string = i 
    split_counts = re.findall('\d+',listingcount_string.text)
    print("list of integers:  ",split_counts)
    this_page = int(split_counts[1])
    total_listings = int(split_counts[-1])
    pages_to_search = -(-total_listings//this_page)
    return pages_to_search 

In [9]:
def make_subsequent_urls(page_count,base_url):
    urls_list = []
    for i in range(2,page_count+1):
        # i will be our page numbers to shoot into the url. 
        next_url = base_url + f"/p{i}"
        urls_list.append(next_url) 
    return urls_list # if it doesn't produce any urls, we moce on.
# https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/p14

In [10]:
base_url 

'https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021'

In [12]:
page_count = number_of_pages(body_soup=body_soup)
urls_list = make_subsequent_urls(page_count=page_count, base_url=base_url)

In [202]:
import pandas as pd
def extract_listing_info(driver,soup_body):
  """ Extract info from all listings on one page of property24.com/to-rent
      
  """
  data = []
  current_url = driver.current_url
  print("The Current_url is ",current_url)

  

  # FIND ALL PRODUCT ITEMS
  info_of_listings = soup_body.find_all(['div',"span","a"],{'class',"p24_content"}) # need to find a waty to get rid of the duplicate 
  image_info = soup_body.find_all(['span','div'],{"class":["js_listingTileImageHolder",'p24_promotedImage']}) # need to find a waty to get rid of the duplicate
  url_info =soup_body.find_all("div",{'class':["p24_regularTile",'p24_promotedTile']}) # still has a duplicate, as with the others, the different one or promoted one only appears on page 1
  # I am assuming that the appearance changes, they proabably sell the space to the propert companies
  print('len(info_of_listings)',len(info_of_listings),"len(image_info)",len(image_info),"len(url_info)",len(url_info))
  print('len(info_of_listings)',len(set(info_of_listings)),"len(image_info)",len(set(image_info)),"len(url_info)",len(set(url_info)))
  # so on page 2 of Greenpoint, suddenly there is one less in the info of listings. 
  # Which means it actually wasn't duplicated for once, but still it causes problems
  # we need to remove the duplicates earlier
  assert len(info_of_listings)== len(image_info)==len(url_info)
  print("BATCH SIZE:", len(info_of_listings))
  

  # Extracting to a dictionary
  counter = 0 
  for i,j,k in zip(info_of_listings,image_info,url_info): 
    # time will tell if the duplicate is giving us problems or if it's the same one every page then we can just delete it at the end 
      d = {'title':'','priceDescription':'','priceAdditionalDescriptor':'', 'suburb':'',
           'number_of_bedrooms':'','number_of_bathrooms':'','number_of_garages':'',"size":"",
           'available_from':'',"url":"",}  
           
      
      # we need an or for the available from or available now flags
      try:
        d['title']                      = i.find(['div','span'],{"class":['p24_title','p24_description']}).text 
        d['priceDescription']           =i.find(['div','span'],{"class":'p24_price'}).text.strip()
        if i.find(['div','span'],{"class":"left rentalTerm p24_rentalTerm"}):
          d['priceAdditionalDescriptor']  = i.find(['div','span'],{"class":"left rentalTerm p24_rentalTerm"}).text
        else: 
          d['priceAdditionalDescriptor'] = None
          
        d['suburb']                     = i.find('span',{"class":"p24_location"}).text

        if i.find("span",{"class":"p24_featureDetails","title":"Bedrooms"}):
          d['number_of_bedrooms']         = i.find("span",{"class":"p24_featureDetails","title":"Bedrooms"}).text.strip()
        else: 
          d['number_of_bedrooms'] = None
        
        if i.find("span",{"class":"p24_featureDetails","title":"Bathrooms"}):
          d['number_of_bathrooms']        = i.find("span",{"class":"p24_featureDetails","title":"Bathrooms"}).text.strip()
        else: d['number_of_bathrooms'] = None
        
        if i.find("span",{"class":"p24_featureDetails","title":"Parking Spaces"}):
          d['number_of_garages']          = i.find("span",{"class":"p24_featureDetails","title":"Parking Spaces"}).text.strip()
        else: d['number_of_garages'] = None
        
        if i.find("span",{"class":"p24_size"}):
          d['size'] = i.find("span",{"class":"p24_size"}).text.strip() # floor size for apartments and houses, commerical property sometimes has
                                                                      #floor size and sometimes has gross-lettable-area. could split in future if needed
        else:
          d["size"] = None 

        if j.find('li',{"class":"p24_availableBadge"}):
          d['available_from']   = j.find('li',{"class":"p24_availableBadge"}).text
        
        else:
          d['available_from'] = None
        
        

        
        
        d['url']              = current_url + "/" + k['data-listing-number']
         
        
      except Exception as e: 
        print(e)
      data.append(d) 
      counter +=1  
  return data 

found it in this one  There us a Showing before this


In [ ]:
extract_listing_info(driver,temp_soup)
# when you get back, keep on with the formula to get the info:
# might need a two stepper for the DIVs
# do the spans first /html/body/div[1]/div[6]/div/div/div[1]/div[2]/div/div

In [ ]:
driver.close()


In [200]:
def main():
    #steps
    all_data = []
    # go to url and get throught the cookie
    driver  = setup_driver()
    # choose the suburb from a list 
    suburbs= ("Sea Point",'Green Point',"Gardens")
    for suburb in suburbs:
        base_url = original_search(driver,suburb)
        soup_body = make_soup(driver) 
        number_of_pages_to_search = number_of_pages(soup_body)
        other_urls_list = make_subsequent_urls(number_of_pages_to_search,base_url)

        #get info from one page 
        data = extract_listing_info(driver, soup_body)
        all_data.extend(data)
        
        for url in other_urls_list:
            driver.get(url)
            next_soup_body  = make_soup(driver)
            data_next = extract_listing_info(driver,next_soup_body)
            all_data.extend(data_next)
        # go bac to the search page 
        time.sleep(5)
        driver.get('https://www.property24.com/to-rent')
        search_bar = driver.find_element(By.XPATH,'//*[@id="token-input-AutoCompleteItems"]')
        search_bar.click()
        search_bar.send_keys(Keys.BACKSPACE)
        
    driver.close()
    df = pd.DataFrame(all_data,index = range(len(all_data)))
    
    return df

In [171]:


driver = setup_driver()
original_search(driver,"Sea Point")
time.sleep(5)
driver.get('https://www.property24.com/to-rent')
# option 1 backspace the previous search and continue,
search_bar = driver.find_element(By.XPATH,'//*[@id="token-input-AutoCompleteItems"]')
search_bar.clear()

<ipython-input-2-41c0e65daecb>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [204]:
# we have two problems: One is that Seapoint is still there when we go back to the origuinal page, Seapoint is still there, the other is that the cookies button only needs to be pressed once
df = main()
df 

<ipython-input-2-41c0e65daecb>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


string:  

                Showing : 1 - 20
of 282


found showing <div class="pull-right">
<span>
                Showing : <span class="p24_bold">1 - 20</span>
of <span class="p24_bold">282</span>
</span>
</div>
string:  






string:  
Download the App




list of integers:   ['1', '20', '282']
The Current_url is  https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021
len(info_of_listings) 22 len(image_info) 22 len(url_info) 22
len(info_of_listings) 21 len(image_info) 21 len(url_info) 22
BATCH SIZE: 22
The Current_url is  https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/p2
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 20 len(image_info) 20 len(url_info) 21
BATCH SIZE: 21
The Current_url is  https://www.property24.com/to-rent/sea-point/cape-town/western-cape/11021/p3
len(info_of_listings) 21 len(image_info) 21 len(url_info) 21
len(info_of_listings) 20 len(image_info) 16 len(url_info) 21
BATCH SIZE: 21
T

,title,priceDescription,priceAdditionalDescriptor,suburb,number_of_bedrooms,number_of_bathrooms,number_of_garages,size,available_from,url
0,3 Bedroom Apartment in Bantry Bay,R 75 000,None,Bantry Bay,3,2.5,2,268 m²,None,https://www.property24.com/to-rent/sea-point/c...
1,2 Bedroom Apartment in Sea Point,R 25 000,None,Sea Point,2,2,1,None,None,https://www.property24.com/to-rent/sea-point/c...
2,1 Bedroom Apartment,R 14 000,None,Sea Point,1,1,1,84 m²,None,https://www.property24.com/to-rent/sea-point/c...
3,2 Bedroom Apartment,R 28 000,None,Sea Point,2,2,1,133 m²,None,https://www.property24.com/to-rent/sea-point/c...
4,2 Bedroom Apartment,R 60 000,None,Sea Point,2,3,1,290 m²,None,https://www.property24.com/to-rent/sea-point/c...
...,...,...,...,...,...,...,...,...,...,...
698,Commercial Property,R 150\n per m²,per m²,Gardens,None,None,None,190 m²,None,https://www.property24.com/to-rent/gardens/cap...
699,Commercial Property,R 51 216,None,Gardens,None,None,None,338 m²,None,https://www.property24.com/to-rent/gardens/cap...
700,Commercial Property,R 7 140,None,Gardens,None,None,None,102 m²,None,https://www.property24.com/to-rent/gardens/cap...
701,Commercial Property,R 7 326,None,Gardens,None,None,None,37 m²,None,https://www.property24.com/to-rent/gardens/cap...


In [ ]:
# tomorrow, The URL doesn't delete the extra one when we add set() overr the list
#2 Need to remove the extra text from the per m2 ones
# then double check
# otherwise think we are ready to move on to pamgolding. 